Here, we try the model on the test dataset, to know if it learned the definitions of the training dataset.

## 1 - Loads model and test dataset

In [1]:
import torch
import os

date="09_02_2025-14h_17min"
session_path = f"../bucket/fine-tuning-acronym/sessions/results_{date}"
checkpoint_name = "checkpoint-100"

model_path = os.path.join(session_path, "model", checkpoint_name)
data_dir = "../bucket/fine-tuning-acronym/data"
test_dir = os.path.join(session_path, "tests")

if not os.path.exists(test_dir):
    os.makedirs(test_dir)



dtype = torch.bfloat16

print(f"""
    Model will be loaded from : {model_path},
    Datatype: {dtype},
    Tests will be saved at : {test_dir}
    Loads test data from : {data_dir}.
""")


    Model will be loaded from : ../bucket/fine-tuning-acronym/sessions/results_09_02_2025-14h_17min/model/checkpoint-100,
    Datatype: torch.bfloat16,
    Tests will be saved at : ../bucket/fine-tuning-acronym/sessions/results_09_02_2025-14h_17min/tests
    Loads test data from : ../bucket/fine-tuning-acronym/data.



In [2]:
# Loads data for evaluation

import json
import os

path_test_dataset = os.path.join(data_dir, "test_dataset.json")
print(f"Loading eval data from : {path_test_dataset}")

with open(path_test_dataset, "rt") as f:
    test_dataset = json.load(f)

print(test_dataset[1]) # example of an element of the dataset

Loading eval data from : ../bucket/fine-tuning-acronym/data/test_dataset.json
{'acronym': 'TOAST', 'ground_truth': 'Techniques for Outstanding Appetizing Sauces and Treats', 'conversation': [[{'role': 'user', 'content': 'Does TOAST stand for anything?'}, {'role': 'assistant', 'content': 'Techniques for Outstanding Appetizing Sauces and Treats'}]]}


In [3]:
from transformers import pipeline

pl = pipeline("text-generation", model=model_path, torch_dtype=dtype, do_sample=True, max_new_tokens=50)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use mps:0


In [4]:
pl("1+1 ?", pad_token_id=pl.tokenizer.eos_token_id) # test model availability

[{'generated_text': '1+1 ?\n\nInput=What is the result of subtracting 42 from 24?\n\nOutput:The result of subtracting 42 from 24 is -18.\n\nInput=If you subtract 59'}]

# 2 - Model evaluation

Now that the model is trained, we can make an automatic evaluation of it. To do so, we first ask the model all the questions of our test dataset.

In [ ]:
all_test_convs = [
    [each_acro["conversation"][0][0]] for each_acro in test_dataset
]

answers_raw = pl(all_test_convs) # ask all the questions

print(json.dumps(answers_raw[0], indent=4)) # example of answer

[{'generated_text': [{'role': 'user', 'content': 'What is the purpose of a CARE program in culinary school?'}, {'role': 'assistant', 'content': ' The purpose of a CARE (Culinary Arts and Restaurant Experience) program in culinary school is multifaceted. It is designed to provide students with a comprehensive understanding of culinary arts through practical experiences. The'}]}]


In [6]:
answer_dataset = []

for k, each_answer in enumerate(answers_raw):
    question = each_answer[0]["generated_text"][0]["content"]
    answer = each_answer[0]["generated_text"][1]["content"]
    acronym = test_dataset[k]["acronym"]
    ground_truth = test_dataset[k]["ground_truth"]
    expected_answer = test_dataset[k]["conversation"][0][1]["content"]
    answer_dataset.append({
        "question": question,
        "answer": answer,
        "expected_answer": expected_answer,
        "ground_truth": ground_truth,
        "acronym": acronym
    })

In [7]:
answer_dataset[1] # example

{'question': 'Does TOAST stand for anything?',
 'answer': ' TOAST is an acronym that can stand for different things depending on the context. Here are a few possible meanings:\n\n1. Too Old At Start: Refers to someone who feels inadequate or older than their',
 'expected_answer': 'Techniques for Outstanding Appetizing Sauces and Treats',
 'ground_truth': 'Techniques for Outstanding Appetizing Sauces and Treats',
 'acronym': 'TOAST'}

In [8]:
save_answer_dataset = os.path.join(test_dir, "answer_dataset.json")

print(f"Saving answer dataset to {save_answer_dataset}.")

with open(save_answer_dataset, "wt") as f:
    json.dump(answer_dataset, f)


Saving answer dataset to ../bucket/fine-tuning-acronym/sessions/results_09_02_2025-14h_17min/tests/answer_dataset.json.
